# Modelamiento NanoCapsulas

Basado en el modelo de Ferrari [1] de la particula reactiva, como se observa en la sigueinte figura

<img src="./imagenes/Esquema_modelo_capsula.png" alt="Escquema modelo nanocapsula de silice" style="width:650px;"/>

 - En la región donde estaria el cascaron de silice solo habria difusión
 - En la region nuclear de la capsula habria tanto reacción con el agente activo como reacción
 
Con base en esto se define una ecuación de transporte para la concentracion de oxígeno de tipo 

$$\frac{\partial C_{O_2}}{\partial t }= D(r)\nabla^2 C_{O_2} + (1-H(r-R_{in}))R_{O_2}$$

Para poder modelar correctamente las nanocapsulas se debe calcular la cantidad de aceite encapsulado en una sola capsula , con el fin de calcular el radio del nucleo reactivo y el grosor del cascaron de silice.

Segun los TGA realizados a las nancapsulas de silice el porcentaje en peso de aceite de linaza encapsulado dentro del cascaron de silice corresponde a 35.46% mientras que el cascarón corresponde a un 46.33%. 

<img src="./imagenes/TGA_Nanocapsulas.png " alt="TGA Nanocapsulas de silice con aceite" style="width:650px;"/>

Dado que se asumira que las nanocpasulas solo estan compuestas de estos dos materiales se va a normalizar con respecto al 81.76% que representan estos dos cantidades, por lo cual el aceite representaria un 43.37% del peso total mientras que el cascaron de silice representaria el 56.63%wt restante. 

Ahora considerando que el diametro de una nanocapsula de silice es aproximadamente 150nm se puede calcular un volumen de $1.7671x10^{-21} m^3$ (asumiendo geometria perfectamente esferica). Sabiendo esto y dado que deseamos encontrar la cantidad de silice y aceite en una nanocapsula, se plantea el siguiente sistema de ecuaciones a solucionar 

> 1. $\frac{w_{Ac}}{w_{Si}+w_{Ac}}=0.4337$ 
> 2. $V_{esf}=\left( \frac{w_{Ac}}{\rho_{Ac}}+\frac{w_{Si}}{\rho_{Si}}\right)$ 

A continuacion se muestra la solución al problema


In [2]:
import numpy as np
from scipy.optimize import fsolve

def sistema_ecuaciones(w):
    "Variables a determinar"
    w_ac=w[0];# masa aceite en kg
    w_si=w[1];# masa silice en kg
    
    "Constantes Conocidas"
    rho_ac=931 #[kg/m3]
    rho_si=2650 #[kg/m3]
    Vf=1.7671e-21 #[m3]

    sol= np.empty((2))
    sol[0] = w_ac/(w_si+w_ac) -0.4337
    sol[1] = ((w_ac/rho_ac)+(w_si/rho_si)-Vf)*1e6
    return sol


wGuess = np.array([1.7671e-15*0.931,1.7671e-15*2.6505])
w = fsolve(sistema_ecuaciones,wGuess)
print(w)

[1.12780677e-18 1.47262387e-18]


A partir del procedimiento anterior se encuentra que el peso de aceite y silice en una capsula debe ser 

> <div style="text-align:center">$w_{Ac}=1.127x10^{-18} kg$ ,$w_{Si}=1.4726x10^{-18} kg$ </div>

ya con esto podemos determinar el radio del centro activo de aceite de linaza asi como el grosor del cascaron de sílice, para esto encontramos el volumen de aceite de linaza usando la densidad del mismo y la masa recien encontrada

> <div style="text-align:center">$V_{Ac}=\frac{w_{Ac}}{\rho_{Ac}}=1.2114x10^{-21} m^3$ </div>

Se asumiendo que el centro de la capsulas es completamente esferico, se determina el radio interno de la capsula, siendo este $6.613x10^{-8}m=66.13nm$. Ya con esta medida se determina el grosor del cascaron de silice restandole al radio total de la capsula el radio del centro de aceite, con lo que se encuentra un valor de $8.87nm$ de grosor. 

Teniendo estos valores, podemos llevar a cabo una simulación de lo que seria el consumo de oxigeno por parte de la nanocapsula y comparar este desempeño con las mediciones experimentales de captura oxígeno en el espacio de cabeza. El dominio computacional de la simulacion estara dividido en dos partes, una seccion correspondiente al cascaron de silice donde solo habra difusion de oxigeno (esta seccion estara dada para $R_{int}<r<R_{ext}$) y la region del nucleo de aceite donde habran tanto difusion como reaccion de oxigeno con el aceite ($r>R_{int}$).

Ahora con el fin de simplificar el problema, se llevará a cabo una simulacion 1D puesto que el sistema posee simetria esferica y la concentracion solo varia de manera radial. De ahi que la ecuacion de transporte a resolver se puede escribir como 

> <div style="text-align:center">$\frac{\partial C_{O_2}}{\partial t }= D(r)\frac{1}{r^2}\frac{\partial}{\partial r} \left(r^2 \frac{\partial C_{O_2}}{\partial r}\right) + (1-H(r-R_{int}))R_{O_2}$</div>

En donde la difusividad en funcion del radio $D(r)$ estara dada por 

  $$ D(r)=   \left\{
\begin{array}{ll}
      D_{Si} & R_{int}<r<R_{ext} \\
      D_{Ac} & r<R_{int} \\
\end{array} 
\right.  $$

Asi mismo la funcion escalón $H(r-R_{int})$ esta definida como 

  $$ H(r-R_{int})=   \left\{
\begin{array}{ll}
      1 & R_{int}<r \\
      0 & r<R_{int} \\
\end{array} 
\right.  $$

y finalmente el término reactivo de la ecuación esta descrito segun la siguinte cinetica de oxidacion

> <div style="text-align:center">2ROOH $\xrightarrow{k_1}$ ROO<sup>$\cdot$</sup> + R<sup>$\cdot$</sup> </div>
> <div style="text-align:center">R<sup>$\cdot$</sup>+ O<sub>2</sub> $\xrightarrow{k_2}$ ROO<sup>$\cdot$</sup> </div>
> <div style="text-align:center">ROO<sup>$\cdot$</sup>+ RH $\xrightarrow{k_3}$ ROOH+R<sup>$\cdot$</sup> </div>
> <div style="text-align:center">2R<sup>$\cdot$</sup> $\xrightarrow{k_4}$ Producto inactivo </div>
> <div style="text-align:center">R<sup>$\cdot$</sup> + ROO<sup>$\cdot$</sup> $\xrightarrow{k_5}$ Producto inactivo </div>   
> <div style="text-align:center">2ROO<sup>$\cdot$</sup> $\xrightarrow{k_6}$ Producto inactivo </div>

Por lo cual $R_{O_2}=-k_2C_{O_2}C_{R^\cdot}$. Ya habiendo definido cada termino de la ecuación de transporte se prodece a realizar la discretización del dominio sobre la cual se va a resolver la ecuación. Para esto la variable continua pasara a ser discreta como $r\xrightarrow{}r_i, 0<i<n+1$ y $t\xrightarrow{}t^j, 0<j<m$. Por otro lado se debe especificar que esquemas de discretizacion se usaran para poder solucionar la ecuacion diferencial, en este caso esta se solucionara por del metodo de lineas (MOL) en el cual se expresa explicitamente las derivdas y funciones que dependen del espacion con el fin de obtener una ecuacion diferencial ordinaria con respecto al tiempo, la cual se va a solucionar a su vez por el metodo de BDF. Ahora bien, para el esquema de discretizacion que se va a utilizar para el termino difusivo de la reaccion se aplicará un esquema central de segundo orden 

$$D(r)\frac{1}{r^2}\frac{\partial}{\partial r} \left(r^2 \frac{\partial C_{O_2}}{\partial r}\right)= D_i\frac{1}{\Delta r^2}\left(\left(1+\frac{\Delta r}{r_i}\right)C_{i+1} -2C_i+\left(1-\frac{\Delta r}{r_i}\right)C_{i-1}\right)$$

Asi mismo la discretizacion del termino reactivo y la funcion escalon sera

$$R_{O_2}=-(1-H(r_i-R_{int}))k_2C_{O_2 ,i}C_{R^\cdot, i}$$

Con esto ultimo, la ecuacion parcial diferencial se puede resscribir como una ecuacion diferencial ordnaria con respecto a las variables de concentracion y espacio discretizados

$$\frac{\partial C_{O_2}}{\partial t }= D_i\frac{1}{\Delta r^2}\left(\left(1+\frac{\Delta r}{r_i}\right)C_{i+1} -2C_i+\left(1-\frac{\Delta r}{r_i}\right)C_{i-1}\right) -(1-H(r_i-R_{int}))k_2C_{O_2 ,i}C_{R^\cdot, i}$$

Finalmente se establecen las condiciones de frontera e iniciales del problema. Las condiciones de frontera seran que la capsula esta expuesta a una concentracion constasnte de oxígeno fuera de la capsula (lo cual asume que la capsula esta permanente expuesta al aire), mientras que para la condicion en elc entro de la esfera sera que no hay flux o flujo de oxigeno dado que este punto representa un eje de simetria del sistema considerado.

> 1. $C_{O_2}(t,r=R_{ext})=C_o$;
> 2. $\frac{\partial C_{O_2}}{\partial r}(t,r=0)=0$

Por otro lado la condicion inicial del sistema sera que inicialmente no habra oxigeno en ninguna parte de la capsula, mientras que la concentracion inicial de las demas especies estara dada por la concentracion que tienen en el aceite de linaza. 

Con el fin de tener en cuenta las condiciones de frontera en la ecuacion diferencial discretizada se aproxima la condicion de frontera 2 segun un esquema central alrededor $C_{O_2,0}$ y se establece una variable fantasma $C_{O_2,-1}$;

$$\frac{\partial C_{O_2}}{\partial r}(t,r=0)=\frac{ C_{O_2,1}-C_{O_2,-1}}{2 \Delta r}=0$$
$$C_{O_2,-1}=C_{O_2,1}$$

Con lo cual se tiene que la ecuacion diferencial que resolver para la variable $C_{O_2,0}$ sera

$$\frac{\partial C_{O_2,0}}{\partial t }= D_0\frac{2}{\Delta r^2}\left(C_{1} -C_0\right) -(1-H_0))k_2C_{O_2 ,0}C_{R^\cdot, 0}$$


In [125]:
from scipy.integrate import solve_ivp, odeint

D_ac=1 #[m2/s]
D_si=2 #[m2/s]
sol_ac=1.58E+07 #[Pa L/mol]
r_ext=7.5e-8 #[m]
r_in=6.613e-8 #[m]
n=100

#Vector de radio discretizado
r=np.linspace(0,r_ext,n)
delta_r=r[1];
#Funcion Escalon 
H=(r_in<r).astype(int);
#Funcion Difusividad
D=D_si*H+D_ac*(1-H)
#constantes de reaccion
k=np.asarray([6.60316288e-11 1e+04 1.29581075e-02 3e+08 1e+07 1.00056782e+04]) #[m3/mol s]

#Vector de inicializacion
C=np.ones([n,5])*np.asarray([0.005*1000,0,0,6.8*1000,0])
C[:,:4] = (C[:,:4].transpose() * (1-H)).transpose()
C=np.reshape(C.transpose(),5*n);
C[-1]=8.57 #[mol/m3];

#Ecuacion Diferencial a solucionar
def ODE(t,C):
    sol= np.empty((np.size(C))); 
    C=np.reshape(C,[5,n]).transpose();
    dC=np.empty([n,5])
    
    dC[:,0]=-2*k[0]*C[:,0]**2+k[2]*C[:,1]*C[:,3];
    dC[:,1]=k[0]*C[:,0]**2+k[1]*C[:,2]*C[:,4]-k[2]*C[:,1]*C[:,3]-k[4]*C[:,2]*C[:,1]-2*k[5]*C[:,1]**2;
    dC[:,2]=k[0]*C[:,0]**2-k[1]*C[:,2]*C[:,4]+k[2]*C[:,1]*C[:,3]-2*k[3]*C[:,2]**2-k[4]*C[:,2]*C[:,1];
    dC[:,3]=-k[2]*C[:,1]*C[:,3];
    dC[0,-1]=D[0]*(2/(delta_r**2))*(C[1,-1]-C[0,-1])-(1-H[0])*k[1]*C[0,-1]*C[0,2];
    dC[1:-1,-1]=D[1:-1]*(1/(delta_r**2))*((1+delta_r/r[1:-1])*C[:-2,-1]-2*C[1:-1,-1]+(1-delta_r/r[1:-1])*C[2:,-1])-(1-H[1:-1])*k[2]*C[1:-1,-1]*C[1:-1,2];
    dC[-1,-1]=0
    
    dC=np.reshape(C,5*n);
    return dC

t=np.linspace(0,3600*300,1)
sol = odeint(ODE,C, t, ml=2, mu=2)

SyntaxError: invalid syntax (<ipython-input-125-ef9d2ef21ba9>, line 18)

In [111]:
T=298
R=8.314
k=np.asarray([4.09e7*np.exp(-101.5e3/(R*T)),1e4,3.52e3*np.exp(-31e3/(R*T)),3e8,1e7,3.05e12*np.exp(-48.4e3/(R*T))])
print (k)

[6.60316288e-11 1.00000000e+04 1.29581075e-02 3.00000000e+08
 1.00000000e+07 1.00056782e+04]


In [137]:
#Ecuacion Diferencial a solucionar

D_ac=0.4e-9 #[m2/s]
sol_ac=1.58E+03 #[Pa m3/mol]

#Vector de radio discretizado
r=np.linspace(0,1.25e-4,n)
delta_r=r[1];

#Vector de inicializacion
C=np.ones([n,5])*np.asarray([0.005*1000,0,0,6.8*1000,0])
C=np.reshape(C.transpose(),5*n);
C[0]=15640.8/(2*sol_ac) #[mol/m3];
#constantes de reaccion
k=np.asarray([6.60316288e-11, 1e+04,1.29581075e-02, 3e+08, 1e+07, 1.00056782e+04]) #[m3/mol s]



def ODE_2(t,C):
    sol= np.empty((np.size(C))); 
    C=np.reshape(C,[5,n]).transpose();
    dC=np.empty([n,5])
    
    dC[:,0]=-2*k[0]*C[:,0]**2+k[2]*C[:,1]*C[:,3];
    dC[:,1]=k[0]*C[:,0]**2+k[1]*C[:,2]*C[:,4]-k[2]*C[:,1]*C[:,3]-k[4]*C[:,2]*C[:,1]-2*k[5]*C[:,1]**2;
    dC[:,2]=k[0]*C[:,0]**2-k[1]*C[:,2]*C[:,4]+k[2]*C[:,1]*C[:,3]-2*k[3]*C[:,2]**2-k[4]*C[:,2]*C[:,1];
    dC[:,3]=-k[2]*C[:,1]*C[:,3];
    dC[-1,-1]=D_ac*(2/(delta_r**2))*(C[-2,-1]-C[-1,-1])-k[1]*C[-1,-1]*C[-1,2];
    dC[1:-1,-1]=D_ac*(1/(delta_r**2))*(C[:-2,-1]-2*C[1:-1,-1]+C[2:,-1])-k[2]*C[1:-1,-1]*C[1:-1,2];
    dC[0,-1]=0
    
    dC=np.reshape(C,5*n);
    return dC
    
t=np.linspace(0,3600,3600+1)
sol = solve_ivp(ODE_2, [0,3600],C,method='BDF',t_eval=t,rtol=1e-11,atol=1e-15);
    

<ipython-input-137-438367cc3ca7>:26: RuntimeWarning: overflow encountered in multiply
  dC[:,2]=k[0]*C[:,0]**2-k[1]*C[:,2]*C[:,4]+k[2]*C[:,1]*C[:,3]-2*k[3]*C[:,2]**2-k[4]*C[:,2]*C[:,1];
<ipython-input-137-438367cc3ca7>:26: RuntimeWarning: overflow encountered in subtract
  dC[:,2]=k[0]*C[:,0]**2-k[1]*C[:,2]*C[:,4]+k[2]*C[:,1]*C[:,3]-2*k[3]*C[:,2]**2-k[4]*C[:,2]*C[:,1];
<ipython-input-137-438367cc3ca7>:25: RuntimeWarning: overflow encountered in multiply
  dC[:,1]=k[0]*C[:,0]**2+k[1]*C[:,2]*C[:,4]-k[2]*C[:,1]*C[:,3]-k[4]*C[:,2]*C[:,1]-2*k[5]*C[:,1]**2;
<ipython-input-137-438367cc3ca7>:25: RuntimeWarning: overflow encountered in subtract
  dC[:,1]=k[0]*C[:,0]**2+k[1]*C[:,2]*C[:,4]-k[2]*C[:,1]*C[:,3]-k[4]*C[:,2]*C[:,1]-2*k[5]*C[:,1]**2;
<ipython-input-137-438367cc3ca7>:25: RuntimeWarning: invalid value encountered in subtract
  dC[:,1]=k[0]*C[:,0]**2+k[1]*C[:,2]*C[:,4]-k[2]*C[:,1]*C[:,3]-k[4]*C[:,2]*C[:,1]-2*k[5]*C[:,1]**2;
<ipython-input-137-438367cc3ca7>:25: RuntimeWarning: overflow enc

ValueError: array must not contain infs or NaNs

[[5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00
  5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000000e+00 5.0000